## THIS NOTEBOOK IS WRITTEN TO RUN ON GOOGLE COLAB

In [1]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

np.random.seed(42)
tf.random.set_seed(42)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [15]:
!git clone https://github.com/JRAlbanese/DS4002-Project3

Cloning into 'DS4002-Project3'...
remote: Enumerating objects: 3258, done.
remote: Counting objects: 100% (1652/1652), done.
remote: Compressing objects: 100% (1642/1642), done.
remote: Total 3258 (delta 16), reused 1641 (delta 5), pack-reused 1606 (from 1)
Receiving objects: 100% (3258/3258), 95.69 MiB | 26.69 MiB/s, done.
Resolving deltas: 100% (17/17), done.
Updating files: 100% (3214/3214), done.
Filtering content: 100% (3202/3202), 239.31 MiB | 2.89 MiB/s, done.


In [16]:
%mv /content/DS4002-Project3/Resized_image_data/Valid/* /content/DS4002-Project3/Resized_image_data/
%mv /content/DS4002-Project3/Resized_image_data/Training/* /content/DS4002-Project3/Resized_image_data/

In [23]:
import pandas as pd

#df = pd.read_csv('/content/combined_annotations.coco.csv')
df = pd.read_csv('/content/DS4002-Project3/Annotations/combined_annotations.coco.csv')
df = df.drop('info__year', axis = 1)
df = df.drop('info__version', axis = 1)
df = df.drop('info__url', axis = 1)#
df = df.drop('info__description', axis = 1)
df = df.drop('info__contributor', axis = 1)
df = df.drop('info__date_created', axis = 1)
df = df.drop('licenses__id', axis = 1)
df = df.drop('licenses__url', axis = 1)
df = df.drop('licenses__name', axis = 1)

In [44]:
print(df.head(10))

   categories__id        categories__name categories__supercategory  \
0             0.0             pedestrians                      none   
1             1.0  children_wo_disability               pedestrians   
2             2.0   elderly_wo_disability               pedestrians   
3             3.0          non_vulnerable               pedestrians   
4             4.0         with_disability               pedestrians   
5             NaN                     NaN                       NaN   
6             NaN                     NaN                       NaN   
7             NaN                     NaN                       NaN   
8             NaN                     NaN                       NaN   
9             NaN                     NaN                       NaN   

   images__id  images__license  \
0         0.0              1.0   
1         1.0              1.0   
2         2.0              1.0   
3         3.0              1.0   
4         4.0              1.0   
5         5.0 

In [64]:
categories = df[['categories__id', 'categories__name']]
image_paths = df[['images__file_name']]
image_paths = '/content/DS4002-Project3/Resized_image_data/' + image_paths
image_paths = image_paths['images__file_name'].tolist()
classes_annotation = df['annotations__category_id'].tolist()

bounding_boxes = list(zip(df['annotations__bbox__001'], df['annotations__bbox__002'], df['annotations__bbox__003'], df['annotations__bbox__004']))
#print(df.head(10))

print(len(image_paths))
print(len(bounding_boxes))
print(len(classes_annotation))




4623
4623
4623


In [13]:
import cv2
import numpy as np
from tensorflow.keras.applications.mobilenet import preprocess_input

def load_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = preprocess_input(image)  # Normalize the image as required by MobileNet
    return image

def resize_bboxes(bboxes, original_size, target_size=(224, 224)):
    # Adjust bounding boxes to the new image size
    h, w = original_size
    new_h, new_w = target_size
    scale_x = new_w / w
    scale_y = new_h / h
    bboxes_resized = []
    for bbox in bboxes:
        xmin, ymin, xmax, ymax = bbox
        bboxes_resized.append([
            int(xmin * scale_x),
            int(ymin * scale_y),
            int(xmax * scale_x),
            int(ymax * scale_y),
        ])
    return bboxes_resized

In [66]:
images = []
bboxes = []
classes = []

for idx, img_path in enumerate(image_paths):
    try:
        print(img_path)
        img = load_image(img_path)
        images.append(img)
        bboxes.append(bounding_boxes[idx])
        classes.append(classes_annotation[idx])
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")

images = np.array(images)
bboxes = np.array(bboxes)
classes = np.array(classes)


Streaming output truncated to the last 5000 lines.
>  - Expected 'filename' to be a str or path-like object

nan
Error loading image nan: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object

nan
Error loading image nan: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object

nan
Error loading image nan: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object



In [67]:
print(images.shape)
print(bboxes.shape)
print(classes.shape)

(1601, 224, 224, 3)
(1601, 4)
(1601,)


In [69]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_bboxes, test_bboxes, train_classes, test_classes = train_test_split(images, bboxes, classes, test_size=0.2)

In [73]:
from tensorflow.keras.utils import to_categorical
num_classes = 5
train_classes = to_categorical(train_classes, num_classes=num_classes)
test_classes = to_categorical(test_classes, num_classes=num_classes)

In [74]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

#Mobilenet shape
input_shape = (224, 224, 3)
num_classes = 5

# instantiating model base
MNbackbone = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
MNbackbone.trainable = False

# output layers bounding box regression and class prediction
x = layers.GlobalAveragePooling2D()(MNbackbone.output)
x = layers.Dense(1024, activation='relu')(x)
bbox_output = layers.Dense(4, activation='sigmoid', name='bbox')(x)  # Using sigmoid to scale between 0 and 1
class_output = layers.Dense(num_classes, activation='softmax', name='class')(x)

# Combine the outputs into a single model
MobileNetmodel = Model(inputs=MNbackbone.input, outputs=[bbox_output, class_output])


In [77]:
MobileNetmodel.compile(optimizer='adam',
              loss={'bbox': 'mse', 'class': 'categorical_crossentropy'},  # Loss for bbox and classification
              metrics={'bbox': 'accuracy', 'class': 'accuracy'})

MobileNetmodel.fit(train_images, [train_bboxes, train_classes], epochs=30, batch_size=16)

Epoch 1/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - bbox_accuracy: 0.4912 - bbox_loss: 73329.2734 - class_accuracy: 0.6240 - class_loss: 1.3004 - loss: 73330.5781
Epoch 2/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - bbox_accuracy: 0.4935 - bbox_loss: 73329.2734 - class_accuracy: 0.7889 - class_loss: 0.5255 - loss: 73329.8047
Epoch 3/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - bbox_accuracy: 0.4910 - bbox_loss: 73329.2734 - class_accuracy: 0.8160 - class_loss: 0.4776 - loss: 73329.7578
Epoch 4/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - bbox_accuracy: 0.4868 - bbox_loss: 73329.2734 - class_accuracy: 0.6814 - class_loss: 0.7876 - loss: 73330.0703
Epoch 5/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - bbox_accuracy: 0.4860 - bbox_loss: 73329.2734 - class_accuracy: 0.7574 - class_loss: 0.6788 - loss: 73329.9609
Epoch 6/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - bbox_accuracy: 0.4895 - bbox_loss: 73329.2734 - class_accuracy: 0.8453 - class_loss: 0.4185 - loss: 73329.7031
Epoch 7/30
80/8

In [78]:
test_loss, bbox_loss, class_loss, bbox_accuracy, class_accuracy = MobileNetmodel.evaluate(test_images, [test_bboxes, test_classes])

print(f"Test loss: {test_loss}")
print(f"Bounding Box Loss: {bbox_loss}")
print(f"Classification Loss: {class_loss}")
print(f"Bounding Box Accuracy: {bbox_accuracy}")
print(f"Classification Accuracy: {class_accuracy}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 162ms/step - bbox_accuracy: 0.4991 - bbox_loss: 73897.8906 - class_accuracy: 0.3280 - class_loss: 3.1968 - loss: 74655.8125
Test loss: 63629.71875
Bounding Box Loss: 59098.19140625
Classification Loss: 3.180340528488159
Bounding Box Accuracy: 0.46417444944381714
Classification Accuracy: 0.3208722770214081
